In [81]:
import boto3
import pandas as pd
import geopandas as gpd
import gdal
import yaml
from yaml import Loader
from shapely.geometry import mapping
import logging
import os
import numpy as np
from osgeo import gdal_array
from skimage.filters import sobel
from sklearn import preprocessing
from skimage.segmentation import slic, watershed
from osgeo import ogr
import cv2 as cv
from skimage.future import graph
import subprocess
from osgeo import osr
from sklearn import preprocessing
# from skimage.color import rgb2gra

# import shapely.geometry import mapping

def parse_catalog_from_s3(bucket, prefix, catalog_name):
    """
    read bucket, prefix from yaml.
    arg:
        bucket: Name of the S3 bucket.
        prefix: prefix for yaml file
        catalog_name: name of catalog file
    return:
        'catalog' pandas object
    """
    s3 = boto3.client('s3')
    obj = s3.get_object(Bucket=bucket, Key='{}/{}'.format(prefix, catalog_name))
    catalog = pd.read_csv(obj['Body'], sep=" ")
    return catalog

def parse_yaml_from_s3(bucket, prefix):
    """
    read bucket, prefix from yaml.
    arg:
        bucket: Name of the S3 bucket.
        prefix: the name for yaml file
    return:
        yaml object
    """
    s3 = boto3.resource('s3')
    obj = s3.Bucket(bucket).Object(prefix).get()['Body'].read()
    return yaml.load(obj)

def get_colrow_geojson(foc_gpd_tile, left_corner_x, left_corner_y, per_tile_width, logger):
    extent_geojson = mapping(foc_gpd_tile['geometry'])
    center_x = (extent_geojson['bbox'][0] + extent_geojson['bbox'][2]) / 2
    center_y = (extent_geojson['bbox'][1] + extent_geojson['bbox'][3]) / 2
    row = int(round((left_corner_y - center_y +  per_tile_width / 2) / per_tile_width)) - 1
    col = int(round((center_x - left_corner_x + per_tile_width / 2) / per_tile_width)) - 1
    return (col, row)

def weight_mean_color(graph, src, dst, n):
    """Callback to handle merging nodes by recomputing mean color.

    The method expects that the mean color of `dst` is already computed.

    Parameters
    ----------
    graph : RAG
        The graph under consideration.
    src, dst : int
        The vertices in `graph` to be merged.
    n : int
        A neighbor of `src` or `dst` or both.

    Returns
    -------
    data : dict
        A dictionary with the `"weight"` attribute set as the absolute
        difference of the mean color between node `dst` and `n`.
    """

    diff = graph.node[dst]['mean color'] - graph.node[n]['mean color']
    diff = np.linalg.norm(diff)
    return {'weight': diff}


def merge_mean_color(graph, src, dst):
    """Callback called before merging two nodes of a mean color distance graph.

    This method computes the mean color of `dst`.

    Parameters
    ----------
    graph : RAG
        The graph under consideration.
    src, dst : int
        The vertices in `graph` to be merged.
    """
    graph.node[dst]['total color'] += graph.node[src]['total color']
    graph.node[dst]['pixel count'] += graph.node[src]['pixel count']
    graph.node[dst]['mean color'] = (graph.node[dst]['total color'] /
                                     graph.node[dst]['pixel count'])

def gdal_save_file_tif_3bands(outpath, r, g, b, gdal_type, trans, proj, rows, cols):
    """
    save file
    Parameters
    ----------
    outpath : full outputted path
    r : the first band (numpy array)
    g : the second band (numpy array)
    b : the third band (numpy array)
    gdal_type: gdal type
    trans: transform coefficients
    proj: projection
    Returns
    -------
    TRUE OR FALSE
    """
    outdriver = gdal.GetDriverByName("GTiff")
    outdata = outdriver.Create(outpath, rows, cols, 3, gdal_type)
    if outdata is None:
        return False
    outdata.GetRasterBand(1).WriteArray(r)
    outdata.FlushCache()
    outdata.GetRasterBand(2).WriteArray(g)
    outdata.FlushCache()
    outdata.GetRasterBand(3).WriteArray(b)
    outdata.FlushCache()
    outdata.SetGeoTransform(trans)
    outdata.FlushCache()
    outdata.SetProjection(proj)
    outdata.FlushCache()
    outdata = None
    return True


def gdal_save_file_tif_1bands(outpath, array, gdal_type, trans, proj, rows, cols):
    """
    save file
    Parameters
    ----------
    outpath : full outputted path
    array : numpy array to be saved
    gdal_type: gdal type
    trans: transform coefficients
    proj: projection
    Returns
    -------
    TRUE OR FALSE
    """
    outdriver = gdal.GetDriverByName("GTiff")
    outdata = outdriver.Create(outpath, rows, cols, 3, gdal_type)
    if outdata is None:
        return False
    outdata.GetRasterBand(1).WriteArray(array)
    outdata.FlushCache()
    outdata.SetGeoTransform(trans)
    outdata.FlushCache()
    outdata.SetProjection(proj)
    outdata.FlushCache()
    outdata = None
    return True

tile_id = str(486641)
# 486255
# 486650
# 486641
season = 'GS'
tmp_pth = '/tmp'
buf = 11
s3_bucket = 'activemapper'
aoi = '1'
left_corner_x = -17.541
left_corner_y = 37.54
per_tile_width = 0.005 * 10 # 0.005 degree is the width of cells, 1 tile has 10*10 cells


log_path = '%s/log/segmenter_%s.log' % (os.environ['HOME'], str(aoi))
logging.basicConfig(filename=log_path, filemode='w', level=logging.INFO)
logger = logging.getLogger(__name__)


with open("/home/ubuntu/source/segmenter_config.yaml", 'r') as yaml_obj:
    params = yaml.safe_load(yaml_obj)['segmenter']
    
prefix = params['planet_prefix']
planet_directory = params['planet_directory']
prob_directory = params['prob_directory']
tiles_geojson_path = params['tile_geojson_path']
mmu = params['mmu']
prob_threshold = params['prob_threshold']
dry_lower_ordinal = params['dry_lower_ordinal']  # 2018/12/01
dry_upper_ordinal = params['dry_upper_ordinal']  # 2019/02/28
wet_lower_ordinal = params['wet_lower_ordinal']  # 2018/05/01
wet_upper_ordinal = params['wet_upper_ordinal']  # 2018/09/30

working_dir  = '/tmp'
verbose = False
buf = 11 # we have a buf for each 2000 * 2000 tile
proj = ''

uri_tile = "s3://{}/{}/{}".format(s3_bucket, prefix, tiles_geojson_path)
gpd_tile = gpd.read_file(uri_tile)
if gpd_tile is None:
    logger.error("reading geojson tile '{}' failed". format(uri_tile))
foc_gpd_tile = gpd_tile[gpd_tile['tile'] == int(tile_id)]
(tile_col, tile_row) = get_colrow_geojson(foc_gpd_tile, left_corner_x, left_corner_y, per_tile_width, logger)
if season is 'OS':
    uri_composite_gdal = "/vsis3/{}/{}/{}/tile{}_{}_{}.tif".format(s3_bucket, planet_directory, season, tile_id, dry_lower_ordinal, dry_upper_ordinal)
else:
    uri_composite_gdal = "/vsis3/{}/{}/{}/tile{}_{}_{}.tif".format(s3_bucket, planet_directory, season, tile_id, wet_lower_ordinal, wet_upper_ordinal)
uri_prob_gdal = "/vsis3/{}/{}/image_c{}_r{}.tif".format(s3_bucket, prob_directory, str(tile_col), str(tile_row))

In [82]:
uri_prob_gdal

'/vsis3/activemapper/classified-images/1_whole_test/image_c300_r532.tif'

In [76]:
# a quick way to read image as numpy array
array_composite = gdal_array.LoadFile(uri_composite_gdal)
array_prob = gdal_array.LoadFile(uri_prob_gdal)

# get channels, cols, rows
[nchannels, cols, rows] = array_composite.shape

# temporal change for dealing with Ron's Rf probability image
# array_prob = array_prob[buf:cols - buf, buf:rows - buf]

# STEP 1
# meanshift algorithm to smooth image and filter out noise
B1, b2, b3, b4 = array_composite

# scale to int8 for opencv processing
min_max_scaler = preprocessing.MinMaxScaler(feature_range=(0, 255))

# get min and max
b2_scale = min_max_scaler.fit_transform(b2.astype(np.float64).reshape(cols * rows, 1)).astype(np.uint8)
b3_scale = min_max_scaler.fit_transform(b3.astype(np.float64).reshape(cols * rows, 1)).astype(np.uint8)
b4_scale = min_max_scaler.fit_transform(b4.astype(np.float64).reshape(cols * rows, 1)).astype(np.uint8)

# keep records for min and max for original bands
b2_min = np.min(b2)
b2_max = np.max(b2)
b3_min = np.min(b3)
b3_max = np.max(b3)
b4_min = np.min(b4)
b4_max = np.max(b4)

mat_norm = cv.merge([b2_scale.reshape(cols, rows), b3_scale.reshape(cols, rows), b4_scale.reshape(cols, rows)])
# meanshift algorithm
dst_norm = cv.pyrMeanShiftFiltering(mat_norm, 5, 5, termcrit=(cv.TERM_CRITERIA_EPS | cv.TERM_CRITERIA_COUNT, 1, 5))

# rescale to int16 (i.e., original band data range)
b2_m, b3_m, b4_m = cv.split(dst_norm)
min_max_scaler = preprocessing.MinMaxScaler(feature_range=(b2_min, b2_max))
b2_filter = min_max_scaler.fit_transform(b2_m.reshape(cols * rows, 1).astype(np.float32))
min_max_scaler = preprocessing.MinMaxScaler(feature_range=(b3_min, b3_max))
b3_filter = min_max_scaler.fit_transform(b3_m.reshape(cols * rows, 1).astype(np.float32))
min_max_scaler = preprocessing.MinMaxScaler(feature_range=(b4_min, b4_max))
b4_filter = min_max_scaler.fit_transform(b4_m.reshape(cols * rows, 1).astype(np.float32))

# mat_filter_rescale = cv.merge([b2_filter.reshape(cols, rows),
#                    b3_filter.reshape(cols, rows),
#                    b4_filter.reshape(cols, rows)])
# r, g, b = cv.split(mat_filter_rescale)

array_filter = np.dstack((b2_filter.reshape(cols, rows),
                                b3_filter.reshape(cols, rows),
                                b4_filter.reshape(cols, rows)))

# save the results for checking
# read metadata info
if verbose is True:
    metadata = gdal.Open(tile_path)
    trans = metadata.GetGeoTransform()
    # proj = metadata.GetProjection()
    outpath = os.path.join(working_dir, 'tile{}_{}_meanshift.tif'.format(tile_id, season))
    gdal_save_file_tif_3bands(outpath, b2_filter.reshape(cols, rows),
                              b3_filter.reshape(cols, rows),
                              b4_filter.reshape(cols, rows),
                              gdal.GDT_Float32, trans, proj, rows, cols)


# STEP 2
# sober filtering + watershed
r_sobel = sobel(b2_filter.reshape(cols, rows))
g_sobel = sobel(b3_filter.reshape(cols, rows))
b_sobel = sobel(b4_filter.reshape(cols, rows))
gradient = r_sobel + g_sobel + b_sobel
gradient_subset = gradient[buf:cols - buf, buf:rows - buf]
# peak_gradient = peak_local_max(-gradient, num_peaks=2400, min_distance=10, indices=False)
# markers = ndi.label(peak_gradient)[0]

# For a 2D image, a connectivity of 1 corresponds to immediate neighbors up, down, left, and right,
# while a connectivity of 2 also includes diagonal neighbors.
segments_watershed = watershed(gradient_subset, markers=2400, connectivity=2, compactness=0).astype(np.int16)

# read metadata info
metadata = gdal.Open(uri_prob_gdal)
trans = metadata.GetGeoTransform()
# proj = metadata.GetProjection()

if verbose is True:
    outpath = os.path.join(working_dir, 'tile{}_{}_meanshift_sober.tif'.format(tile_id, season))
    gdal_save_file_tif_1bands(outpath, gradient_subset, gdal.GDT_Float32, trans, proj, rows - 2 * buf, cols - 2 * buf)
    outpath = os.path.join(working_dir, 'tile{}_{}_meanshift_watershed.tif'.format(tile_id, season))
    gdal_save_file_tif_1bands(outpath, segments_watershed, gdal.GDT_Int16, trans, proj, rows - 2 * buf, cols - 2 * buf)

# segments_slic = slic(arr_filter_rescale, n_segments=2400,compactness=0.1)
# outpath = os.path.join(working_path, 'split_tile1_os_slic.tif')
# gdal_save_file_tif_1bands(outpath, segments_slic, gdal.GDT_Int16, trans, proj, rows, cols)


# STEP 3
# overlapped with prob image, and selected those polygons that have average probability over 0.5
# segments_watershed_merge_sieve = gdal_array.LoadFile(os.path.join(working_dir, tile_id + '_watershed_merge_sieve.tif'))
for i in range(1, np.max(segments_watershed)):
    condition = np.equal(segments_watershed, i)
    prob_condition = np.extract(condition, array_prob)
    if np.mean(prob_condition) < prob_threshold:
        segments_watershed[condition] = 0


if verbose is True:
    outpath = os.path.join(working_dir,  'tile{}_{}_watershed_overlap.tif'.format(tile_id, season))
    gdal_save_file_tif_1bands(outpath, segments_watershed, gdal.GDT_Int16, trans, proj, rows - 2 * buf,
                              cols - 2 * buf)

# STEP 4
# connected small polygons using graph theory + sieving filter
min_max_scaler = preprocessing.MinMaxScaler(feature_range=(0, 1))
array_original_rescale = np.dstack((min_max_scaler.fit_transform(b2.reshape(cols * rows, 1).astype(np.float32))
                                    .reshape(cols, rows),
                                    min_max_scaler.fit_transform(b3.reshape(cols * rows, 1).astype(np.float32))
                                    .reshape(cols, rows),
                                    min_max_scaler.fit_transform(b4.reshape(cols * rows, 1).astype(np.float32))
                                    .reshape(cols, rows)))

b2_filter_withingrid = array_original_rescale[buf:cols - buf, buf:rows - buf, 0]
field_b2_std = np.std(b2_filter_withingrid[array_prob > prob_threshold])
# field_b2_std = np.std(b2_filter_withingrid[arr_prob > prob_threshold])
# nofield_b2_mean = np.mean(b2_filter_withingrid[array_prob <= prob_threshold])
# nofield_b2_std = np.std(b2_filter_withingrid[arr_prob <= prob_threshold])

b3_filter_withingrid = array_original_rescale[buf:cols - buf, buf:rows - buf, 1]
field_b3_std = np.std(b3_filter_withingrid[array_prob > prob_threshold])
# nofield_b3_mean = np.mean(b3_filter_withingrid[array_prob <= threshold])

b4_filter_withingrid = array_original_rescale[buf:cols - buf, buf:rows - buf, 2]
field_b4_std = np.std(b4_filter_withingrid[array_prob > prob_threshold])
# nofield_b4_mean = np.mean(b4_filter_withingrid[array_prob <= prob_threshold])

# calculate norm to adaptively decide merging threshold
# diff_norm = np.linalg.norm([nofield_b2_mean - field_b2_mean, nofield_b3_mean - field_b3_mean,
#                             nofield_b4_mean - field_b4_mean])
diff_norm = np.linalg.norm([field_b2_std, field_b3_std, field_b4_std])

array_original_subset = array_original_rescale[buf:cols - buf, buf:rows - buf]

# assign -9999 so background pixels won't be merged
array_original_subset[segments_watershed == 0] = [-9999, -9999, -9999]

g = graph.rag_mean_color(array_original_subset, segments_watershed,
                         mode='distance')

segments_watershed_merge = graph.merge_hierarchical(segments_watershed, g, thresh=diff_norm/2, rag_copy=False,
                                                    in_place_merge=True,
                                                    merge_func=merge_mean_color,
                                                    weight_func=weight_mean_color)

# note after merging, it sometimes reset polygons id, which cause the id of no field to be not zero any more, the below
# is the function to fix this issue
for i in range(np.max(segments_watershed_merge)):
    condition = np.equal(segments_watershed_merge, i)
    id_condition = np.extract(condition, segments_watershed)
    counts = np.bincount(id_condition)
    if np.argmax(counts) == 0:
        nofield_id = i
        break

if nofield_id is not 0:
    # for nofield id is not 0, switch 0 and nofield id
    segments_watershed_merge[segments_watershed_merge == 0] = 9999
    segments_watershed_merge[segments_watershed_merge == nofield_id] = 0
    segments_watershed_merge[segments_watershed_merge == 9999] = nofield_id


# required to save it for sieve filter
outpath = os.path.join(working_dir, 'tile{}_{}_watershed_overlap_merge.tif'.format(tile_id, season))
gdal_save_file_tif_1bands(outpath, segments_watershed_merge, gdal.GDT_Int16, trans, proj, rows - 2 * buf,
                          cols - 2 * buf)


# sieving filter
cmd = 'gdal_sieve.py -q -st {} -8 {} {}'.format(mmu, os.path.join(working_dir, 'tile{}_{}_watershed_overlap_merge.tif'.format(tile_id, season)),
                                                os.path.join(working_dir, 'tile{}_{}_watershed_overlap_merge_sieve.tif'.format(tile_id, season)))
os.system(cmd)

# STEP 5
# polyganize
outpath = os.path.join(working_dir, 'tile{}_{}_watershed_overlap_merge_sieve.tif'.format(tile_id, season))
src_ds = gdal.Open(outpath)
if src_ds is None:
    print('Unable to open %s' % outpath)

srcband = src_ds.GetRasterBand(1)

srs = osr.SpatialReference()
srs.ImportFromWkt(proj)
dst_layername = os.path.join(working_dir, 'tile{}_{}_watershed_merge_sieve_overlap'.format(tile_id, season))
drv = ogr.GetDriverByName("GeoJSON")
dst_ds = drv.CreateDataSource(dst_layername + ".geojson")
dst_layer = dst_ds.CreateLayer(dst_layername, srs = srs)

fieldName = "id"
fd = ogr.FieldDefn(fieldName, ogr.OFTInteger)
dst_layer.CreateField(fd)
dst_field = dst_layer.GetLayerDefn().GetFieldIndex("id")

gdal.Polygonize(srcband, None, dst_layer, dst_field, [], callback=None)
dst_ds = None # it guaranttee shapefile is successfully created
src_ds = None

# STEP 6
# post-processing
infile = dst_layername + ".geojson"
outfile = os.path.join(working_dir, 'tile{}_{}_seg.geojson'.format(tile_id, season))
command = 'Rscript'
path_script = "Postprocessing.R"
args = [infile, outfile, '0.00005']
if (os.path.isfile(path_script) == False):
    print('Fail to find Postprocessing.R')
# check_output will run the command and store to result
cmd = [command, path_script] + args
x = subprocess.check_output(cmd, universal_newlines=True)

# remove temporal file
if verbose is False:
    outpath = os.path.join(working_dir, 'tile{}_{}_watershed_overlap_merge.tif'.format(tile_id, season))
    os.remove(outpath)
    outpath = os.path.join(working_dir, 'tile{}_{}_watershed_overlap_merge_sieve.tif'.format(tile_id, season))
    os.remove(outpath)
    outpath = os.path.join(working_dir, 'tile{}_{}_watershed_merge_sieve_overlap.geojson'.format(tile_id, season))
    os.remove(outpath)


In [54]:
infile = dst_layername + ".geojson"
outfile = os.path.join(working_dir, 'tile{}_{}_seg.geojson'.format(tile_id, season))
command = 'Rscript'
path_script = "Postprocessing.R"
args = [infile, outfile, '0.00005']
if (os.path.isfile(path_script) == False):
    print('Fail to find Postprocessing.R')
# check_output will run the command and store to result
cmd = [command, path_script] + args
x = subprocess.check_output(cmd, universal_newlines=True)

In [55]:
array_prob
    

array([[48, 42, 38, ..., 67, 63, 59],
       [51, 45, 37, ..., 63, 61, 57],
       [52, 47, 37, ..., 60, 58, 48],
       ...,
       [33, 27, 26, ..., 10, 25, 27],
       [34, 29, 28, ..., 11, 28, 26],
       [30, 31, 27, ..., 29, 26, 29]], dtype=int8)